# PROYECTO MICROCREDITO

In [ ]:
!pip install unidecode
!pip install lazypredict

##Importaciones

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from unidecode import unidecode
import re
from google.colab import drive
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import unicodedata
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB

## Carga de datos

In [ ]:
# Montar drive
drive.mount('/content/drive')

# Desmontar drive inmediatamente
#drive.flush_and_unmount()

In [ ]:
# Guardo el archivo de info formulario como un dataframe
df_unificado_total = pd.read_csv('/content/drive/MyDrive/proyecto_microcreditos/data/processed/df_unificado_total.csv')

In [ ]:
# Configurar la opcion para ver todas las columnas
pd.set_option('display.max_columns', None)

In [ ]:
pd.set_option('display.max_rows', 500)

## EDA

### Análisis y limpieza

In [ ]:
df_unificado_total.info()

In [ ]:
# Convertir suma_ingresos a numerico
df_unificado_total['suma_ingresos'] = pd.to_numeric(df_unificado_total['suma_ingresos'], errors='coerce')

In [ ]:
df_unificado_total.describe(include='all')  # para ver estadísticas generales

**Observacion:** Porqué hay un -1 en antiguedad de ocupacion?

In [ ]:
df_unificado_total['antiguedad_ocupacion'].value_counts()[-1] # Cantidad de valores en -1

In [ ]:
df_unificado_total['antiguedad_ocupacion'] = df_unificado_total['antiguedad_ocupacion'].replace(-1, 0)

#### Distribuciones de variables numericas

In [ ]:
# Obtener los nombres de las columnas con tipos de datos numéricos
num_vars = df_unificado_total.select_dtypes(include=['number']).columns

# Histogramas
df_unificado_total[num_vars].hist(figsize=(15, 12), bins=30)
plt.tight_layout()
plt.show()

# Boxplots para detectar outliers
for col in num_vars:
    sns.boxplot(x=df_unificado_total[col])
    plt.title(f'Distribución de {col}')
    plt.show()

**Observación:** Se eben validar las columnas 'suma_ingresos' y 'antiguedad_ocupacion', ya que presentan valores atipicos muy altos

In [ ]:
# Variables a evaluar
variables = ['suma_ingresos', 'antiguedad_ocupacion']

# Total de registros
total_rows = len(df_unificado_total)

# Función para evaluar outliers
for col in variables:
    print(f"\n Análisis de outliers para: {col}")

    # Calcular percentiles
    p99 = df_unificado_total[col].quantile(0.99)
    p999 = df_unificado_total[col].quantile(0.999)

    # Contar valores mayores
    out_99 = (df_unificado_total[col] > p99).sum()
    out_999 = (df_unificado_total[col] > p999).sum()

    # Mostrar resultados
    print(f"  - Valor percentil 99: {p99:,.2f}")
    print(f"  - Valor percentil 99.9: {p999:,.2f}")
    print(f"  - Registros > P99: {out_99} ({out_99 / total_rows:.2%})")
    print(f"  - Registros > P99.9: {out_999} ({out_999 / total_rows:.2%})")

**Observacion:** Se van a eliminar los que esten por encima del percentil 99, ya que los valores tomados por las variables en ese punto son coherentes

In [ ]:
# Definir umbrales
p99_suma_ingresos = df_unificado_total['suma_ingresos'].quantile(0.99)
p99_antiguedad = df_unificado_total['antiguedad_ocupacion'].quantile(0.99)

# Filtrar registros sin outliers extremos
df_filtrado = df_unificado_total[
    (df_unificado_total['suma_ingresos'] <= p99_suma_ingresos) &
    (df_unificado_total['antiguedad_ocupacion'] <= p99_antiguedad)
]

# Mostrar cuántos registros quedan
print(f"Registros originales: {len(df_unificado_total)}")
print(f"Registros sin outliers (>P99.9): {len(df_filtrado)}")
print(f"Eliminados: {len(df_unificado_total) - len(df_filtrado)} registros ({(1 - len(df_filtrado)/len(df_unificado_total)):.2%})")

#### Distribución de variables categoricas

In [ ]:
# Convertir automáticamente todas las columnas object a category
for col in df_unificado_total.select_dtypes(include='object').columns:
    df_unificado_total[col] = df_unificado_total[col].astype('category')

# Obtener los nombres de las columnas categóricas
cat_vars = df_unificado_total.select_dtypes(include=['category']).columns

In [ ]:
def limpiar_texto(texto):
    if isinstance(texto, str):
        # Quitar acentos
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
        # Pasar a minúsculas y quitar espacios
        return texto.strip().lower()
    return texto  # Dejar valores no string (ej. NaN) sin tocar

In [ ]:
# Aplicar limpieza a cada valor en cada columna categórica
for col in cat_vars:
    df_unificado_total[col] = df_unificado_total[col].apply(limpiar_texto)

In [ ]:
# Convertir columnas categóricas a tipo 'category' después de limpiar
for col in cat_vars:
    df_unificado_total[col] = df_unificado_total[col].astype('category')

In [ ]:
# Graficacion de distribución de variables categoricas
# Conteo de frecuencia
for col in cat_vars:
    if df_unificado_total[col].nunique() <= 10:
        print(df_unificado_total[col].value_counts(normalize=True) * 100)
        sns.countplot(x=col, data=df_unificado_total)
        plt.title(f'Distribución de {col}')
        plt.xticks(rotation=45)
        plt.show()

In [ ]:
for col in cat_vars:
    if df_unificado_total[col].nunique() > 10:
        # Obtener top 10 valores únicos más frecuentes
        top10 = df_unificado_total[col].value_counts().nlargest(10)

        # Crear figura
        plt.figure(figsize=(8, 4))
        sns.barplot(x=top10.values, y=top10.index.astype(str))
        plt.title(f'Top 10 valores en {col}')
        plt.xlabel("Frecuencia")
        plt.ylabel(col)
        plt.tight_layout()
        plt.show()

### Verificacion de valores nulos

In [ ]:
# Verificar nulos: cantidad y porcentaje
nulos = df_unificado_total.isnull().sum()
porcentaje = (nulos / len(df_unificado_total)) * 100

# Crear un DataFrame ordenado por % de nulos
nulos_df = pd.DataFrame({
    'cantidad_nulos': nulos,
    'porcentaje_nulos': porcentaje
}).sort_values(by='porcentaje_nulos', ascending=False)

# Mostrar las columnas con al menos un nulo
nulos_df[nulos_df['cantidad_nulos'] > 0]

#### Enriquecimiento de categoricos con asofondos y mareigua

In [ ]:
# Copia de trabajo
df_copy = df_unificado_total.copy()

# Convertir temporalmente a object
df_copy['empresa'] = df_copy['empresa'].astype('object')
df_copy['eps'] = df_copy['eps'].astype('object')

# ----------- EMPRESA -----------

cond_empresa = df_copy['empresa'].isna() & df_copy['nombre_empleador'].notna()
num_cambios_empresa = cond_empresa.sum()

if df_copy['empresa'].dtype.name == 'category':
    # Obtener categorías únicas que no estén ya incluidas
    nuevas_categorias = df_copy.loc[cond_empresa, 'nombre_empleador'].dropna().unique()
    nuevas_categorias = [cat for cat in nuevas_categorias if cat not in df_copy['empresa'].cat.categories]
    df_copy['empresa'] = df_copy['empresa'].cat.add_categories(nuevas_categorias)

df_copy.loc[cond_empresa, 'empresa'] = df_copy.loc[cond_empresa, 'nombre_empleador']

# ----------- EPS -----------

cond_eps = df_copy['eps'].isna() & df_copy['EPS'].notna()
num_cambios_eps = cond_eps.sum()

if df_copy['eps'].dtype.name == 'category':
    nuevas_categorias_eps = df_copy.loc[cond_eps, 'EPS'].dropna().unique()
    nuevas_categorias_eps = [cat for cat in nuevas_categorias_eps if cat not in df_copy['eps'].cat.categories]
    df_copy['eps'] = df_copy['eps'].cat.add_categories(nuevas_categorias_eps)

df_copy.loc[cond_eps, 'eps'] = df_copy.loc[cond_eps, 'EPS']

# ----------- RESULTADO -----------

print("Cambios realizados:")
print(f" - 'empresa' imputada con 'nombre_empleador': {num_cambios_empresa} registros.")
print(f" - 'eps' imputada con 'EPS': {num_cambios_eps} registros.")

In [ ]:
# Convertir de nuevo a category
df_copy['empresa'] = df_copy['empresa'].astype('category')
df_copy['eps'] = df_copy['eps'].astype('category')

In [ ]:
df_unificado_total[['empresa', 'eps']] = df_copy[['empresa', 'eps']] # Aplicar los cambios

In [ ]:
df_unificado_total.drop(['EPS', 'nombre_empleador'], axis=1, inplace=True)

#### Imputacion de categoricos 'desconocido'

In [ ]:
# Lista de columnas categóricas con nulos
cat_cols_with_nulls = [
    'operador_cel', 'tipo_afiliacion', 'tipo_plan_celular',
    'eps', 'nivel_estudios', 'sector_ocupacion',
    'tamano_empresa', 'empresa', 'horario_contacto'
]

for col in cat_cols_with_nulls:
    df_unificado_total[col] = df_unificado_total[col].cat.add_categories('desconocido')
    df_unificado_total[col] = df_unificado_total[col].fillna('desconocido')

In [ ]:
df_unificado_total.info()

#### Enriquecimiento con datacredito

In [ ]:
# Copia de seguridad
df_unificado_total['tiene_creditos_antes_de_solicitar_original'] = df_unificado_total['tiene_creditos_antes_de_solicitar']
df_unificado_total['tiene_reporte_negativo_antes_de_solicitar_original'] = df_unificado_total['tiene_reporte_negativo_antes_de_solicitar']

# Reemplazo basado en saldo_actual_sum (>0 → "si", ==0 → "no")
cond_credito_valido = df_unificado_total['saldo_actual_sum'].notna()
df_unificado_total.loc[cond_credito_valido, 'tiene_creditos_antes_de_solicitar'] = np.where(df_unificado_total.loc[cond_credito_valido, 'saldo_actual_sum'] > 0,
    'si',
    'no'
)

# Reemplazo basado en cuentas_castigadas (>0 → "SI", ==0 → "NO")
cond_castigado_valido = df_unificado_total['cuentas_castigadas'].notna()
df_unificado_total.loc[cond_castigado_valido, 'tiene_reporte_negativo_antes_de_solicitar'] = np.where(df_unificado_total.loc[cond_castigado_valido, 'cuentas_castigadas'] > 0,
    'si',
    'no'
)

In [ ]:
# Créditos modificados
modificados_credito = (df_unificado_total['tiene_creditos_antes_de_solicitar'] != df_unificado_total['tiene_creditos_antes_de_solicitar_original']).sum()

# Reportes negativos modificados
modificados_reporte = (df_unificado_total['tiene_reporte_negativo_antes_de_solicitar'] != df_unificado_total['tiene_reporte_negativo_antes_de_solicitar_original']).sum()

print(f'Créditos corregidos: {modificados_credito}')
print(f'Reportes negativos corregidos: {modificados_reporte}')

In [ ]:
df_unificado_total.drop(['cuentas_castigadas', 'saldo_actual_sum', 'tiene_creditos_antes_de_solicitar_original', 'tiene_reporte_negativo_antes_de_solicitar_original'], axis=1, inplace=True)

#### Enriquecimiento con asofondos y mareigua

In [ ]:
# Definir ocupaciones válidas para la imputación
ocupaciones_validas = ['empleado/a termino indefinido', 'empleado/a termino fijo renovable']

# Antigüedad: imputar donde falta y se cumple la condición
cond_antiguedad = (
    df_unificado_total['antiguedad_ocupacion'].isna() &
    df_unificado_total['meses_continuos'].notna() &
    df_unificado_total['ocupacion'].isin(ocupaciones_validas)
)

df_unificado_total.loc[cond_antiguedad, 'antiguedad_ocupacion'] = (
    df_unificado_total.loc[cond_antiguedad, 'meses_continuos'] / 12
)

# Ingresos: imputar donde falta y se cumple la condición
cond_ingresos = (
    df_unificado_total['suma_ingresos'].isna() &
    df_unificado_total['ingresos'].notna() #&
   #df_unificado_total['ocupacion'].isin(ocupaciones_validas)
)

df_unificado_total.loc[cond_ingresos, 'suma_ingresos'] = df_unificado_total.loc[cond_ingresos, 'ingresos']

In [ ]:
print("Antigüedad ocupación imputada:", cond_antiguedad.sum())
print("Suma ingresos imputada:", cond_ingresos.sum())

In [ ]:
# Asegurar que las columnas sean numéricas
df_unificado_total['ingresos'] = pd.to_numeric(df_unificado_total['ingresos'], errors='coerce')
df_unificado_total['suma_ingresos'] = pd.to_numeric(df_unificado_total['suma_ingresos'], errors='coerce')

# Calcular diferencia
df_unificado_total['diferencia'] = df_unificado_total['suma_ingresos'] - df_unificado_total['ingresos']

# Ocupaciones válidas
ocupaciones_validas = ['empleado/a termino indefinido', 'empleado/a termino fijo renovable']

# Asegurar que ocupacion sea string para comparar bien
df_unificado_total['ocupacion'] = df_unificado_total['ocupacion'].astype(str)

# Condición robusta
cond = (
    df_unificado_total['ingresos'].notna() &
    df_unificado_total['suma_ingresos'].notna() &
    (df_unificado_total['suma_ingresos'] > df_unificado_total['ingresos'] * 1.5) &
    df_unificado_total['ocupacion'].isin(ocupaciones_validas)
)

# Candidatos
candidatos = df_unificado_total.loc[cond, ['suma_ingresos', 'ingresos', 'ocupacion', 'diferencia']]
candidatos = candidatos.sort_values('diferencia', ascending=False)

# Mostrar el registro más extremo
registro_max = candidatos.head(5)
print(" Registro con mayor exceso de suma_ingresos vs ingresos:")
print(registro_max)

In [ ]:
# Eliminarlo del DataFrame principal
indice_a_eliminar = registro_max.index[0]
df_unificado_total = df_unificado_total.drop(index=indice_a_eliminar)
print(f"\n Registro eliminado: índice {indice_a_eliminar}")

In [ ]:
df_unificado_total.drop(['meses_continuos', 'ingresos','diferencia'], axis=1, inplace=True)

In [ ]:
df_unificado_total.info()

#### Imputacion de numericos faltantes con knn vecinos

In [ ]:
# Columnas numéricas a escalar e imputar
columnas_knn = ['suma_ingresos', 'antiguedad_ocupacion']

# Extraer las columnas (pueden tener NaNs)
df_knn = df_unificado_total[columnas_knn]

# Escalar los datos (fit_transform devuelve un array NumPy sin índice)
scaler = StandardScaler()
df_knn_scaled = scaler.fit_transform(df_knn)

# Imputar con KNN (también devuelve un array NumPy)
imputer = KNNImputer(n_neighbors=3)
df_knn_imputed_scaled = imputer.fit_transform(df_knn_scaled)  # Imputa en escala estandarizada

# Invertir el escalado para regresar a escala original
df_knn_imputed = scaler.inverse_transform(df_knn_imputed_scaled)

# Convertir a DataFrame **preservando el índice original**
df_knn_imputed = pd.DataFrame(df_knn_imputed, columns=columnas_knn, index=df_knn.index)

# Asignar las columnas imputadas de vuelta al DataFrame original, usando .loc para preservar índices
df_unificado_total.loc[:, columnas_knn] = df_knn_imputed

In [ ]:
df_unificado_total.info()

In [ ]:
# Variables objet a category
df_unificado_total['ocupacion'] = df_unificado_total['ocupacion'].astype('category')

In [ ]:
# Cambios de variables numericas
df_unificado_total['valor_credito'] = df_unificado_total['valor_credito'].astype('int32')
df_unificado_total['plazo_solicitado'] = df_unificado_total['plazo_solicitado'].astype('int8')
df_unificado_total['estado_general'] = df_unificado_total['estado_general'].astype('int8')
df_unificado_total['suma_ingresos'] = df_unificado_total['suma_ingresos'].astype('int32')
df_unificado_total['estrato'] = df_unificado_total['estrato'].astype('int8')
df_unificado_total['antiguedad_ocupacion'] = df_unificado_total['antiguedad_ocupacion'].astype('float32')

## Codificación de variables categoricas

### Codificación de variables binarias

In [ ]:
# Columnas con valores binarios tipo "sí"/"no"
columnas_binarias = [
    'cotiza_seguridad',
    'tiene_reporte_negativo_antes_de_solicitar',
    'tiene_creditos_antes_de_solicitar'
]

# Mapear "sí" -> 1, "no" -> 0 (ignora mayúsculas si es necesario)
df_unificado_total[columnas_binarias] = df_unificado_total[columnas_binarias].apply(
    lambda col: col.str.lower().map({ 'si': 1, 'no': 0})
)

### Codificacion de variables ordinales

In [ ]:
# Verificacion de valores unicos
columnas_a_verificar = ['periocidad_pago', 'nivel_estudios', 'tamano_empresa']

for col in columnas_a_verificar:
    print(f"Valores únicos en '{col}':")
    print(df_unificado_total[col].unique())
    print("-" * 50)

In [ ]:
# Contar los valores 'desconocido' en cada columna
print(df_unificado_total['nivel_estudios'].value_counts().get('desconocido', 0))
print(df_unificado_total['tamano_empresa'].value_counts().get('desconocido', 0))

In [ ]:
#-----------------Empresa---------------------
# Copiar la columna
df_empresas = df_unificado_total[['empresa']].copy()

# Normalización
def normalizar_empresa(nombre):
    if pd.isnull(nombre):
        return "desconocido"

    nombre = str(nombre).lower()
    nombre = unidecode(nombre)  # quitar tildes
    nombre = re.sub(r'[^\w\s]', '', nombre)  # eliminar puntuación
    nombre = re.sub(r'\s+', ' ', nombre).strip()  # espacios dobles
    # Eliminar palabras comunes que no aportan valor
    comunes = ['sa', 'sas', 'ltda', 'sucursal', 'empresa', 'de', 'los', 'las', 'y', 'cia', 'ltda', 'the', 's.a.s', 's.a']
    palabras = nombre.split()
    palabras = [p for p in palabras if p not in comunes]
    return ' '.join(palabras)

# Aplicar la función
df_empresas['empresa_normalizada'] = df_empresas['empresa'].apply(normalizar_empresa)

# Agrupación por frecuencia
frecuentes = df_empresas['empresa_normalizada'].value_counts().nlargest(100).index
df_empresas['empresa_reducida'] = df_empresas['empresa_normalizada'].apply(
    lambda x: x if x in frecuentes else 'otros'
)

In [ ]:
frecuentes

In [ ]:
# Agregar al dataset original
df_unificado_total['empresa_reducida'] = df_empresas['empresa_reducida']

In [ ]:
# Hacer empresa = empresa_reducida
df_unificado_total['empresa'] = df_unificado_total['empresa_reducida']

In [ ]:
#-----------------Tamaño Empresa-------------------------------
# Obtener la moda del tamaño de empresa por cada empresa_reducida
modas_por_empresa = (
    df_unificado_total[df_unificado_total['tamano_empresa'] != 'desconocido']
    .groupby('empresa_reducida')['tamano_empresa']
    .agg(lambda x: x.mode().iloc[0])  # usamos la moda más frecuente
)

# Crear una función para imputar
def imputar_tamano_empresa(row):
    if row['tamano_empresa'] != 'desconocido':
        return row['tamano_empresa']
    empresa = row['empresa_reducida']
    return modas_por_empresa.get(empresa, 'desconocido')  # si no hay moda, se queda como 'desconocido'

# Aplicar la función fila por fila
df_unificado_total['tamano_empresa_imputado'] = df_unificado_total.apply(imputar_tamano_empresa, axis=1)

In [ ]:
# Eliminar empresa reducida
df_unificado_total.drop(['empresa_reducida'], axis=1, inplace=True)

In [ ]:
df_unificado_total['tamano_empresa_imputado'].value_counts()

In [ ]:
df_unificado_total['tamano_empresa'] = df_unificado_total['tamano_empresa_imputado']
df_unificado_total.drop(['tamano_empresa_imputado'], axis=1, inplace=True)

In [ ]:
#----------Nivel de estudios-----------
# Calcular la moda (valor más común)
moda_estudios = df_unificado_total.loc[
    df_unificado_total['nivel_estudios'] != 'desconocido', 'nivel_estudios'
].mode().iloc[0]

# Reemplazar 'desconocido' con la moda
df_unificado_total['nivel_estudios'] = df_unificado_total['nivel_estudios'].replace('desconocido', moda_estudios)

In [ ]:
#---------------Mapeo de variables ordinales---------------------
# Diccionarios de mapeo ordinal
map_periocidad_pago = {
    'semanal': 0,
    'quincenal': 1,
    'mensual': 2
}

map_nivel_estudios = {
    'primaria': 0,
    'secundaria': 1,
    'tecnico / tecnologo': 2,
    'universitario': 3,
    'posgrado': 4
}

map_tamano_empresa = {
    'de 1 a 10': 0,
    'de 11 a 25': 1,
    'de 26 a 50': 2,
    'de 51 a 100': 3,
    'de 100 a 500': 4,
    'mas de 500': 5
}

# Aplicar los mapeos
df_unificado_total['periocidad_pago'] = df_unificado_total['periocidad_pago'].map(map_periocidad_pago)
df_unificado_total['nivel_estudios'] = df_unificado_total['nivel_estudios'].map(map_nivel_estudios)
df_unificado_total['tamano_empresa'] = df_unificado_total['tamano_empresa'].map(map_tamano_empresa)

In [ ]:
# Cambios de tipo de datos de variables ordinales
df_unificado_total['periocidad_pago'] = df_unificado_total['periocidad_pago'].astype('int8')
df_unificado_total['nivel_estudios'] = df_unificado_total['nivel_estudios'].astype('int8')
df_unificado_total['tamano_empresa'] = df_unificado_total['tamano_empresa'].astype('int8')

### Codificación one hot directa

In [ ]:
# Diccionario para renombrar las categorías de estado_civil
mapeo_estado_civil = {
    'soltera/o': 'soltero',
    'casada/o': 'casado',
    'divorciada/o': 'divorciado',
    'viuda/o': 'viudo',
    'union libre': 'union libre'  # Esta categoría no cambia
}

# Reemplazar las categorías en la columna estado_civil
df_unificado_total['estado_civil'] = df_unificado_total['estado_civil'].replace(mapeo_estado_civil)

In [ ]:
# Lista de columnas categóricas a codificar
columnas_onehot = ['tipo_pago', 'tipo_afiliacion', 'tipo_plan_celular', 'estado_civil']

# Realizar codificación one-hot
df_encoded = pd.get_dummies(df_unificado_total, columns=columnas_onehot)

In [ ]:
# Actualizar el DataFrame original con las nuevas columnas codificadas
df_unificado_total = df_encoded.copy()

### Codificaciones de variables que requieren reduccion de categorias

In [ ]:
#----------Ocupacion------------------
df_unificado_total['ocupacion'].value_counts(normalize=True) * 100 #valores unicos de ocupacion y su porcentaje

In [ ]:
# Diccionario para mapear las categorías de ocupacion a las nuevas categorías
mapeo_ocupacion = {
    'empleado/a termino indefinido': 'empleado_estable',
    'empleado/a termino fijo renovable': 'empleado_estable',
    'independiente': 'independiente',
    'empleado/a por servicios': 'empleado_no_estable',
    'empleado/a temporal': 'empleado_no_estable',
    'pensionado/a': 'pensionado',
    'estudiante': 'otros',
    'empleado/a medio tiempo': 'otros',
    'desempleado/a': 'otros'
}

# Reemplazar las categorías en la columna ocupacion
df_unificado_total['ocupacion'] = df_unificado_total['ocupacion'].replace(mapeo_ocupacion)

# Realizar codificación one-hot
df_encoded_ = pd.get_dummies(df_unificado_total, columns=['ocupacion'])

In [ ]:
# Actualizar el DataFrame original
df_unificado_total = df_encoded_.copy()

In [ ]:
df_unificado_total

In [ ]:
#----------horario_contacto------------------
df_unificado_total['horario_contacto'].value_counts(normalize=True) * 100 #valores unicos y su cantidad

In [ ]:
def resumir_horario_contacto(horario):
    if pd.isna(horario) or str(horario).strip() == 'desconocido':
        return 'desconocido'

    horario = str(horario).lower().strip()

    # Flexible / todo el día
    if any(p in horario for p in ['todo el día', 'todo el dia', 'cualquier', 'a cualquier hora', '24 horas', 'disponible']):
        return 'flexible'

    # Mañana
    if any(p in horario for p in ['mañana', 'manana', 'en la mañana', 'en la manana']):
        return 'manana'
    if re.search(r'\b(7|8|9|10|11)([:h]?)(\s?am)?\b', horario):
        return 'manana'

    # Tarde
    if any(p in horario for p in ['tarde', 'en la tarde']):
        return 'tarde'
    if re.search(r'\b(12|13|14|15|16|17|18|19)([:h]?)(\s?pm)?\b', horario):
        return 'tarde'

    # Horarios tipo "8 a 5", "8 AM - 5 PM"
    if re.search(r'8\s*(a|-)\s*5', horario):
        return 'horario_laboral'

    return 'otro'

In [ ]:
df_unificado_total['horario_contacto'] = df_unificado_total['horario_contacto'].apply(resumir_horario_contacto)
df_unificado_total['horario_contacto'].value_counts(normalize=True) * 100

In [ ]:
# Realizar codificación one-hot
df_unificado_total = pd.get_dummies(df_unificado_total, columns=['horario_contacto'])

In [ ]:
df_unificado_total.head()

In [ ]:
#----------fuente_origen_visita------------------
df_unificado_total['fuente_origen_visita'].value_counts(normalize=True) * 100 #valores unicos y su porcentaje

In [ ]:
# agrupamiento.
agrupaciones_fov = {
    'datacredito_oferta_1': 'datacredito',
    'datacredito_oferta_2': 'datacredito',
    'datacredito_oferta_3': 'datacredito',
    'facebook1': 'facebook',
    'facebook2': 'facebook',
    'facebook_campain': 'facebook',
    'instagram_campain': 'instagram',
    'adwords1': 'adwords',
    'sms1': 'sms',
    'sms2': 'sms',
    'sms3': 'sms',
    'sms4': 'sms',
    'sms6': 'sms',
    'sms7': 'sms',
    'sms9': 'sms',
    'noticiadocampo.com':'otros',
    'referido':'otros',
    'rd station':'otros',
    'servy leads':'otros',
    'leads_global':'otros',
    'techkiro.com':'otros',
    'tarjetaahora.com':'otros',
    'affinity':'otros',
    'sms':'otros'
}

df_unificado_total['fuente_origen_visita'] = df_unificado_total['fuente_origen_visita'].replace(agrupaciones_fov)
df_unificado_total['fuente_origen_visita'].value_counts(normalize=True) * 100

In [ ]:
# Realizar codificación one-hot
df_unificado_total = pd.get_dummies(df_unificado_total, columns=['fuente_origen_visita'])

In [ ]:
#----------operador_cel------------------
df_unificado_total['operador_cel'].value_counts(normalize=True) * 100 #valores unicos y su cantidad

In [ ]:
# agrupamiento.
agrupaciones_movil = {
    'virgin mobile': 'otro',
    'avantel': 'otro',
    'etb': 'otro',
    'movil exito': 'otro',
    'uff movil': 'otro',
    'desconocido': 'otro'
}

df_unificado_total['operador_cel'] = df_unificado_total['operador_cel'].replace(agrupaciones_movil)

In [ ]:
# Realizar codificación one-hot
df_unificado_total = pd.get_dummies(df_unificado_total, columns=['operador_cel'])

In [ ]:
#----------sector_ocupacion------------------
df_unificado_total['sector_ocupacion'].value_counts(normalize=True) * 100 #valores unicos y su porcentaje

In [ ]:
mapeo_sector_ocupacion = {
    'comercio': 'servicios',
    'servicios': 'servicios',
    'financiero': 'servicios',
    'industria y manufactura': 'industria',
    'construccion': 'industria',
    'salud': 'salud_educacion',
    'educacion': 'salud_educacion',
    'entidades del estado': 'sector_publico',
    'servicios publicos': 'sector_publico',
    'tecnologia': 'servicios',
    'otros': 'otros',
    'transporte': 'otros',
    'agropecuario': 'otros',
    'desconocido': 'otros'
}
df_unificado_total['sector_ocupacion'] = df_unificado_total['sector_ocupacion'].replace(mapeo_sector_ocupacion)

In [ ]:
# Realizar codificación one-hot
df_unificado_total = pd.get_dummies(df_unificado_total, columns=['sector_ocupacion'])

In [ ]:
#----Eliminar 'ciudad_cedula'----------------
df_unificado_total.drop(['ciudad_cedula'], axis=1, inplace=True)

In [ ]:
#----------ciudad------------------
df_unificado_total['ciudad'].value_counts(normalize=True).head(20) * 100 #valores unicos y su porcentaje

In [ ]:
# Extraer solo la parte izquierda del guion en la columna 'ciudad'
df_unificado_total['ciudad_reducida'] = df_unificado_total['ciudad'].str.split(' - ').str[0]

# Definir umbral para mantener ciudades (por ejemplo, 1%)
umbral = 1.9

# Calcular porcentajes de frecuencia
porcentajes_ciudades = df_unificado_total['ciudad_reducida'].value_counts(normalize=True) * 100

# Identificar ciudades con frecuencia >= umbral
ciudades_mantener = porcentajes_ciudades[porcentajes_ciudades >= umbral].index

# Reemplazar ciudades con frecuencia < umbral por 'otras'
df_unificado_total['ciudad_reducida'] = df_unificado_total['ciudad_reducida'].where(
    df_unificado_total['ciudad_reducida'].isin(ciudades_mantener), 'otras'
)

In [ ]:
df_unificado_total['ciudad'] = df_unificado_total['ciudad_reducida']
df_unificado_total.drop(['ciudad_reducida'], axis=1, inplace=True)

In [ ]:
# Realizar codificación one-hot
df_unificado_total = pd.get_dummies(df_unificado_total, columns=['ciudad'])

In [ ]:
#----------eps------------------
df_unificado_total['eps'].value_counts(normalize=True).head(20) * 100 #valores unicos y su porcentaje

In [ ]:
# Definir umbral para mantener categorías de eps (5%)
umbral_eps = 5

# Calcular porcentajes de frecuencia para eps
porcentajes_eps = df_unificado_total['eps'].value_counts(normalize=True) * 100

# Identificar categorías de eps con frecuencia >= umbral
eps_mantener = porcentajes_eps[porcentajes_eps >= umbral_eps].index

# Si eps es una columna categórica, añadir 'otras' a las categorías si no existe
if df_unificado_total['eps'].dtype.name == 'category':
    if 'otras' not in df_unificado_total['eps'].cat.categories:
        df_unificado_total['eps'] = df_unificado_total['eps'].cat.add_categories(['otras'])

# Crear una nueva columna eps_reducida para evitar modificar eps directamente
df_unificado_total['eps_reducida'] = df_unificado_total['eps'].where(
    df_unificado_total['eps'].isin(eps_mantener), 'otras'
)

# Codificación one-hot para eps_reducida
df_encoded = pd.get_dummies(df_unificado_total, columns=['eps_reducida'], prefix=['eps'])

# Eliminar la columna 'otras' si existe
if 'eps_otras' in df_encoded.columns:
    df_encoded = df_encoded.drop(columns=['eps_otras'])

In [ ]:
#----Eliminar eps y eps_reducida----------------
df_unificado_total.drop(['eps', 'eps_reducida'], axis=1, inplace=True)

In [ ]:
#----------empresa------------------
df_unificado_total['empresa'].value_counts(normalize=True).head(20) * 100 #valores unicos y su porcentaje

**Observacion:** Dado que empresa presenta una alta cardinalidad y la categoria dominante es otros con el 83%, hacer una agrupacion seria equivalente a la variable sector, se decide eliminar la variable

In [ ]:
#----Eliminar empresa----------------
df_unificado_total.drop(['empresa', 'nivel_estudios'], axis=1, inplace=True)

In [ ]:
df_unificado_total.info()

In [ ]:
df_unificado_total['estado_general'].value_counts()

## Guardar el dataset unificado y codificado

In [ ]:
# Especificar la ruta completa del archivo
ruta_archivo = '/content/drive/MyDrive/proyecto_microcreditos/data/processed/df_unificado_codificado.csv'
# Guardar el DataFrame en un archivo CSV
df_unificado_total.to_csv(ruta_archivo, index=False)

## Partición de datos

In [ ]:
# Separar variables
X = df_unificado_total.drop(columns=['estado_general'])  # variable objetivo
y = df_unificado_total['estado_general']

In [ ]:
# Partición del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Analisis de caracteristicas

In [ ]:
#  Random Forest para importancia de variables
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
# Importancia de variables
importancias = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

In [ ]:
importancias.sort_values(ascending = False)*100

In [ ]:
# Calcular la importancia como porcentaje
importancias_porcentaje = importancias * 100

# Seleccionar las características con importancia >= 0.9%
caracteristicas_seleccionadas = importancias_porcentaje[importancias_porcentaje >= 0.9].index

# Crear nuevo conjunto de datos con esas características
X_train_seleccionado = X_train[caracteristicas_seleccionadas]
X_test_seleccionado = X_test[caracteristicas_seleccionadas]  # si tienes test set

## Correlacion de caracteristicas

In [ ]:
# Correlación entre las variables numéricas más importantes
top_vars = X_train_seleccionado.columns
corr_matrix = X_train_seleccionado[top_vars].corr()

# Mostrar solo triángulo inferior
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    annot_kws={"size": 8},   # Tamaño de los números en el mapa
    cmap='coolwarm',
    center=0,
    fmt=".2f"
)
plt.title("Correlación entre variables importantes (triángulo inferior)")
plt.xticks(rotation=90, fontsize=8)  # Tamaño del texto en el eje X
plt.yticks(rotation=0, fontsize=8)   # Tamaño del texto en el eje Y
plt.tight_layout()
plt.show()

**Observaciones:**   
- Se identifica que tipo_pago_sustitucion y tipo_pago_pago_total tienen un 0.91 de correlacion, y de acuerdo a el analisis de caracteristicas tipo_pago_pago_total tiene menor importancia, por lo tanto es la variable a eliminar.
-Tambien que existe una correlacion de 1 entre tipo_plan_celular_prepago y tipo_plan_celular_postpago, y de acuerdo a el analisis de caracteristicas tipo_plan_celular_postpago tiene menor importancia, por lo tanto es la variable a eliminar.


In [ ]:
# ------- eliminar tipo_plan_celular_postpago y tipo_pago_pago_total por correlacion con otras variables
X_train_seleccionado.drop(['tipo_plan_celular_postpago', 'tipo_pago_pago total'], axis=1, inplace=True)
X_test_seleccionado.drop(['tipo_plan_celular_postpago', 'tipo_pago_pago total'], axis=1, inplace=True)

## Selección del modelo a usar

In [ ]:
print("Número de columnas:", X_train_seleccionado.shape[1])
print("Número de filas:", X_train_seleccionado.shape[0])

In [ ]:
X_train_seleccionado.info()

In [ ]:
# Especificar la ruta completa del archivo x_train
ruta_archivo = '/content/drive/MyDrive/proyecto_microcreditos/data/processed/X_train_sel.csv'
X_train_seleccionado.to_csv(ruta_archivo, index=False)

# Especificar la ruta completa del archivo x_test
ruta_archivo = '/content/drive/MyDrive/proyecto_microcreditos/data/processed/X_test_sel.csv'
X_test_seleccionado.to_csv(ruta_archivo, index=False)

# Especificar la ruta completa del archivo y_train
ruta_archivo = '/content/drive/MyDrive/proyecto_microcreditos/data/processed/y_train.csv'
y_train.to_csv(ruta_archivo, index=False)

# Especificar la ruta completa del archivo y_test
ruta_archivo = '/content/drive/MyDrive/proyecto_microcreditos/data/processed/y_test.csv'
y_test.to_csv(ruta_archivo, index=False)

In [ ]:
"""# LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train_seleccionado, X_test_seleccionado, y_train, y_test)

# Ver resultados ordenados por ROC AUC o F1-score si están disponibles
print(models.sort_values(by="F1 Score", ascending=False))  # o "ROC AUC"

# Ver clasificación detallada de uno de los modelos
from sklearn.ensemble import RandomForestClassifier
rf_cl = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_cl.fit(X_train_seleccionado, y_train)
y_pred = rf_cl.predict(X_test_seleccionado)
print(classification_report(y_test, y_pred))"""

In [ ]:
models_to_use = {
    'LogisticRegression': LogisticRegression(),
    'RandomForestClassifier': RandomForestClassifier(),
    'RidgeClassifier': RidgeClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'BernoulliNB': BernoulliNB(),
    'LGBMClassifier': LGBMClassifier()
}

results = []

for name, model in models_to_use.items():
    try:
        model.fit(X_train_seleccionado, y_train)
        y_pred = model.predict(X_test_seleccionado)
        f1 = f1_score(y_test, y_pred, average="weighted")
        results.append({
            "Model": name,
            "F1 Score": f1
        })
    except Exception as e:
        print(f"{name} failed: {e}")

df_results = pd.DataFrame(results)
print(df_results.sort_values(by="F1 Score", ascending=False))